In [1]:
import sys
import struct
import argparse

import numpy as np
import openhd as hd
import pandas as pd
from sklearn.model_selection import train_test_split


/home/hpc/iwi3/iwi3083h/.local/lib/python3.8/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [2]:
!rm -rf ~/.openhd

In [3]:
df = pd.read_csv("data/NYC_Taxi/NYC_cleaned.csv")

In [4]:
y = df['trip_duration']
X = df.drop(['trip_duration'],axis=1)

In [5]:
# X = np.array(X, dtype = np.float32)

# y = np.array(y, dtype = np.float32)


X = np.array(X)

y = np.array(y)

In [6]:
# y[0]= 5
y[1]= 6
y[2]= 7
# y[3]= 8
# y[4]= 9
# y[5]= 10
# y[6]= 11
# y[7] = 12
# y[8]= 13
# y[9]= 14
# y[10]= 15
# y[11]= 16
# y[12]= 17
# y[13]= 18
# y[14]= 19
# y[15] = 20

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [8]:
X = X[:X.shape[0]//10]
y = y[:y.shape[0]//10]

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.3)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((100664, 23), (43142, 23), (100664,), (43142,))

In [10]:
# y_train = np.array(y_train, dtype = np.int32)
# y_test = np.array(y_test, dtype = np.int32)

In [11]:
n_classes = len(set(y))
n_classes

7

In [12]:
N = X_train.shape[0]
F = X_train.shape[1]
N, F

(100664, 23)

In [13]:
Q = 10
D = 10000
hd.init(D=D, context=globals())


In [14]:
# Allocated Memory

(N * D * np.dtype(np.float32).itemsize) / (1024*1024*1024)

3.750026226043701

In [15]:
@hd.run
def create_random_bases():
    id_base = hd.draw_random_hypervector()
    level_base = hd.draw_random_hypervector()
    return id_base, level_base


@hd.run
def create_ids(F, id_base):
    id_hvs = hd.hypermatrix(F) # np.zeros(F, N) (not the empty list) 
    for f in range(F):
        id_hvs[f] = hd.permute(id_base, f)

    return id_hvs

@hd.run
def create_levels(Q, level_base):
    level_hvs = hd.hypermatrix(Q+1) # np.zeros((Q+1), N) (not the empty list)
    for q in range(Q+1):
        idx = int(q/float(Q) * D) / 2
        level_hvs[q] = hd.flip(level_base, idx)
        level_hvs[q] = hd.shuffle(level_hvs[q], 0)

    return level_hvs

@hd.run
def retrain(class_hvs, hv_matrix, labels, N, n_classes):
    search_results = hd.search(class_hvs, hv_matrix)

    for idx in range(N):
        if search_results[idx] != labels[idx]:
            class_hvs[labels[idx]] += hv_matrix[idx]
            class_hvs[search_results[idx]] -= hv_matrix[idx]

    return class_hvs
@hd.run
def single_pass(hv_matrix, labels, N, n_classes):
    class_hvs = hd.hypermatrix(n_classes)

    for idx in range(N):
        class_hvs[labels[idx]] += hv_matrix[idx]

    return class_hvs

@hd.run
def assoc_search(class_hvs, hv_matrix_tst):
    ret = hd.search(class_hvs, hv_matrix_tst)
    return ret

def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


In [16]:
with hd.utils.timing("Base hypervectors"):
    id_base, level_base = create_random_bases()
    id_hvs = create_ids(F, id_base)
    level_hvs = create_levels(Q, level_base)


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [__ARG__id_base : __one__]
  <class '_ast.Assign'> :  [__ARG__level_base : __one__]

{'id_base': 'hypervec_type', 'level_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__level_base': 'float*', 'id_base_22b19319': <class 'float'>, 'level_base_5541502b': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [__ARG__id_hvs : __ARG__id_base]

{'f': <class 'int'>, 'F': <class 'int'>, 'id_hvs': 'hypermat_type', 'id_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__id_hvs': 'float*', 'id_base_f_16a56737': <class 'float'>, 'id_hvs_f_00177a23': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [level_hvs_q_7acb546c : __ARG__level_base]
  <class '_ast.Assign'> :  [__ARG__level_hvs : level_hvs_q_7acb546c]

{'idx': <class 'int'>, 'q': 'int', 'Q': <class 'int'>, 'D': 

In [17]:
def preprocesser(
        org_feature, cnv_feature, # Predefined argument (single feature)
        Q, level_hvs, id_hvs): # arguments passed by args
    cnv_feature = int(org_feature * Q)


def encoder(
        input_features, output_hypervector, # Predefined arguments
        Q, level_hvs, id_hvs): # arguments passed by args
    for f in range(F):
        output_hypervector += level_hvs[input_features[f]] * id_hvs[f]

In [18]:
with hd.utils.timing("Encode training"):
    hv_matrix = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = X_train,
            preprocess_function = preprocesser # optional
            )

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

In [19]:
with hd.utils.timing("Encode testing"):
    hv_matrix_tst = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = X_test,
            preprocess_function = preprocesser # optional
            )


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

In [20]:
type(y_train)

numpy.ndarray

In [21]:
y_train.shape

(100664,)

In [22]:
# with hd.utils.timing("Single pass"):
class_hvs = single_pass(hv_matrix, y_train, N, n_classes)
class_hvs.debug_print_values()

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + __ARG__hv_matrix]

{'idx': <class 'int'>, 'N': <class 'int'>, 'class_hvs': 'hypermat_type', 'labels': 'np_int_array_type', 'hv_matrix': 'hypermat_type', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', '__ARG__class_hvs': 'float*', '__ARG__hv_matrix': 'float*', 'hv_matrix_idx_ead026f6': <class 'float'>, 'labels_idx_b9c77ce4': <class 'int'>, 'class_hvs_labels_idx_df62a0b1': <class 'float'>}


In [23]:
RETRAIN_ITERATIONS = 1000
SHOW_STEP_RESULT = True
for it in range(RETRAIN_ITERATIONS):
#     with hd.utils.timing("Retrain itereation: %d" % it):
    print("Retrain Iteration: ", it)
    class_hvs = retrain(class_hvs, hv_matrix, y_train, N, n_classes)

    if SHOW_STEP_RESULT and y_test is not None:
        validate(y_test, hd.search(class_hvs, hv_matrix_tst).to_numpy())

Retrain Iteration:  0
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [hv_matrix_idx_ead026f6 : __ARG__hv_matrix]
    <class '_ast.If'> : 
      then : 
        <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + hv_matrix_idx_ead026f6]
        <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + hv_matrix_idx_ead026f6]
      else : 

{'idx': <class 'int'>, 'N': <class 'int'>, 'search_results': 'np_int_array_type', 'labels': 'np_int_array_type', 'class_hvs': 'hypermat_type', 'hv_matrix': 'hypermat_type', '__ARG__search_results': 'int*', '__ARG__search_results__STRIDE__': 'const int', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', '__ARG__class_hvs': 'float*', '__ARG__hv_matrix': 'float*', 'search_results_idx_b85d35f6': <class 'int'>, 'labels_idx_b9c77ce4': <class 'int'>, 'hv_matrix_idx_ead026f6': <class 'float'>, 'class_hvs_labels_idx_df62a0b1': <class 'float'>, 'class_hvs_sea

6929 43142 16.060915117518892
Retrain Iteration:  138
7685 43142 17.813267813267814
Retrain Iteration:  139
5136 43142 11.904872282230773
Retrain Iteration:  140
9300 43142 21.55671966992722
Retrain Iteration:  141
0 43142 0.0
Retrain Iteration:  142
7954 43142 18.43679013490334
Retrain Iteration:  143
7577 43142 17.56293171387511
Retrain Iteration:  144
45 43142 0.10430670808029298
Retrain Iteration:  145
9245 43142 21.429233693384635
Retrain Iteration:  146
4718 43142 10.935978860507161
Retrain Iteration:  147
2718 43142 6.3001251680496955
Retrain Iteration:  148
7666 43142 17.769227203189466
Retrain Iteration:  149
63 43142 0.1460293913124102
Retrain Iteration:  150
6644 43142 15.400305966343703
Retrain Iteration:  151
9873 43142 22.88489175281628
Retrain Iteration:  152
3283 43142 7.609753836168931
Retrain Iteration:  153
4232 43142 9.809466413239997
Retrain Iteration:  154
5891 43142 13.654907051133467
Retrain Iteration:  155
7600 43142 17.616244031338372
Retrain Iteration:  156
5

5318 43142 12.326734968244402
Retrain Iteration:  294
5255 43142 12.180705576931992
Retrain Iteration:  295
5939 43142 13.766167539752445
Retrain Iteration:  296
5338 43142 12.373093505168976
Retrain Iteration:  297
11 43142 0.025497195308516064
Retrain Iteration:  298
5112 43142 11.849242037921282
Retrain Iteration:  299
10073 43142 23.34847712206203
Retrain Iteration:  300
5073 43142 11.758842890918363
Retrain Iteration:  301
5399 43142 12.51448704278893
Retrain Iteration:  302
9500 43142 22.020305039172964
Retrain Iteration:  303
0 43142 0.0
Retrain Iteration:  304
2746 43142 6.3650271197441
Retrain Iteration:  305
1657 43142 3.8408047842010107
Retrain Iteration:  306
8632 43142 20.008344536646426
Retrain Iteration:  307
5117 43142 11.860831672152425
Retrain Iteration:  308
5627 43142 13.042974363729082
Retrain Iteration:  309
5020 43142 11.63599276806824
Retrain Iteration:  310
5340 43142 12.377729358861435
Retrain Iteration:  311
8688 43142 20.138148440035234
Retrain Iteration:  3

5620 43142 13.02674887580548
Retrain Iteration:  450
5545 43142 12.852904362338325
Retrain Iteration:  451
4532 43142 10.504844467108617
Retrain Iteration:  452
6453 43142 14.957581938714014
Retrain Iteration:  453
8355 43142 19.366278800241066
Retrain Iteration:  454
0 43142 0.0
Retrain Iteration:  455
7929 43142 18.378841963747625
Retrain Iteration:  456
4858 43142 11.260488618979185
Retrain Iteration:  457
5946 43142 13.782393027676045
Retrain Iteration:  458
9780 43142 22.66932455611701
Retrain Iteration:  459
0 43142 0.0
Retrain Iteration:  460
4958 43142 11.49228130360206
Retrain Iteration:  461
4895 43142 11.346251912289649
Retrain Iteration:  462
5105 43142 11.833016549997682
Retrain Iteration:  463
6523 43142 15.119836817950025
Retrain Iteration:  464
3560 43142 8.25181957257429
Retrain Iteration:  465
1895 43142 4.392471373603449
Retrain Iteration:  466
10809 43142 25.054471280886375
Retrain Iteration:  467
3467 43142 8.036252375875018
Retrain Iteration:  468
5378 43142 12.46

3632 43142 8.418710305502758
Retrain Iteration:  606
27 43142 0.0625840248481758
Retrain Iteration:  607
9727 43142 22.546474433266887
Retrain Iteration:  608
7766 43142 18.00101988781234
Retrain Iteration:  609
4827 43142 11.188632886746094
Retrain Iteration:  610
1320 43142 3.0596634370219276
Retrain Iteration:  611
6207 43142 14.387371934541745
Retrain Iteration:  612
5196 43142 12.043947893004498
Retrain Iteration:  613
9099 43142 21.090816373835242
Retrain Iteration:  614
0 43142 0.0
Retrain Iteration:  615
5698 43142 13.20754716981132
Retrain Iteration:  616
6117 43142 14.17875851838116
Retrain Iteration:  617
4780 43142 11.079690324973344
Retrain Iteration:  618
6320 43142 14.649297668165593
Retrain Iteration:  619
6009 43142 13.928422418988456
Retrain Iteration:  620
5792 43142 13.42543229335682
Retrain Iteration:  621
8303 43142 19.245746604237173
Retrain Iteration:  622
4672 43142 10.82935422558064
Retrain Iteration:  623
1378 43142 3.194103194103194
Retrain Iteration:  624
4

2575 43142 5.968661629038988
Retrain Iteration:  761
7944 43142 18.413610866441054
Retrain Iteration:  762
9436 43142 21.871957721014322
Retrain Iteration:  763
4702 43142 10.898892030967502
Retrain Iteration:  764
0 43142 0.0
Retrain Iteration:  765
45 43142 0.10430670808029298
Retrain Iteration:  766
8878 43142 20.57855454081869
Retrain Iteration:  767
4967 43142 11.513142645218116
Retrain Iteration:  768
6887 43142 15.963562189977285
Retrain Iteration:  769
7459 43142 17.28941634602012
Retrain Iteration:  770
1124 43142 2.605349775161096
Retrain Iteration:  771
4944 43142 11.459830327754856
Retrain Iteration:  772
7466 43142 17.30564183394372
Retrain Iteration:  773
5286 43142 12.252561309165083
Retrain Iteration:  774
8117 43142 18.81461221083863
Retrain Iteration:  775
752 43142 1.7430809883640073
Retrain Iteration:  776
5640 43142 13.073107412730053
Retrain Iteration:  777
6711 43142 15.555607065041027
Retrain Iteration:  778
10621 43142 24.61870103379537
Retrain Iteration:  779


5866 43142 13.596958879977747
Retrain Iteration:  917
3220 43142 7.4637244448565205
Retrain Iteration:  918
9995 43142 23.167678828056186
Retrain Iteration:  919
5747 43142 13.321125585276528
Retrain Iteration:  920
4950 43142 11.473737888832227
Retrain Iteration:  921
4154 43142 9.628668119234158
Retrain Iteration:  922
4794 43142 11.112141300820547
Retrain Iteration:  923
1607 43142 3.724908441889574
Retrain Iteration:  924
7615 43142 17.6510129340318
Retrain Iteration:  925
6072 43142 14.074451810300866
Retrain Iteration:  926
9 43142 0.020861341616058595
Retrain Iteration:  927
7634 43142 17.69505354411015
Retrain Iteration:  928
5758 43142 13.346622780585044
Retrain Iteration:  929
5880 43142 13.629409855824951
Retrain Iteration:  930
1920 43142 4.450419544759168
Retrain Iteration:  931
8925 43142 20.687497102591443
Retrain Iteration:  932
3603 43142 8.351490426962126
Retrain Iteration:  933
5321 43142 12.333688748783088
Retrain Iteration:  934
6916 43142 16.03078206851792
Retrain

In [26]:
print(validate(y_test, hd.search(class_hvs, hv_matrix_tst).to_numpy()))

5104 43142 11.830698623151454
None


In [25]:
n_classes

7